Model download link: https://github.com/OlafenwaMoses/ImageAI/releases/download/essentials-v5/resnet50_coco_best_v2.1.0.h5/

In [1]:
from imageai.Detection import ObjectDetection
import os

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath("models/resnet50_coco_best_v2.1.0.h5")
detector.loadModel()

Detecting objects from thumbnails

In [2]:
import pickle

objects_detected_w_proba = dict()
dir_name = "images"
dir_filenames = os.listdir(dir_name)[:5]
images_left = len(dir_filenames)

for filename in dir_filenames:
    if filename == ".gitignore":
        continue

    # detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename, output_type="array")[1]
    detections = detector.detectObjectsFromImage(input_image=f"{dir_name}/" + filename,
                                                 output_image_path=f"{dir_name}/new_" + filename)

    objects_detected_w_proba[filename] = []

    for eachObject in detections:
        objects_detected_w_proba[filename].append((eachObject["name"], eachObject["percentage_probability"]))
        # print(filename, (eachObject["name"], eachObject["percentage_probability"]))

    images_left -= 1


    if images_left % 100 == 0:
        print("images left:", images_left)
        with open("data/thumbnails_objects_dict.pkl", "wb") as f:
            pickle.dump(objects_detected_w_proba, f)

images left: 4
images left: 3
images left: 2
images left: 1
images left: 0


Loading detections from pickle file

In [1]:
import pickle

# detecions_dict = dict()
with open("data/thumbnails_objects_dict.pkl", "rb") as f:
    objects_detected_w_proba = dict(pickle.load(f))
print("processed images:", len(objects_detected_w_proba.keys()))

processed images: 16432


Counting detected objects

In [2]:
objects_detected = dict()
counts = dict()

for key in objects_detected_w_proba.keys():
    objects_detected[key] = dict()

    for name, _ in objects_detected_w_proba[key]:
        objects_detected[key][name] = objects_detected[key].get(name, 0) + 1
        counts[name] = counts.get(name, 0) + 1

counts = dict(reversed(sorted(counts.items(), key=lambda item: item[1])))
top_popular_objects = list(counts.keys())[:10]

# print("objects counts", objects_detected)
print("counts", counts)
print("10 most common objects:", top_popular_objects)

counts {'person': 26206, 'car': 889, 'cell phone': 717, 'tie': 605, 'bottle': 359, 'cup': 275, 'tv': 267, 'chair': 267, 'sports ball': 258, 'dog': 203, 'truck': 203, 'clock': 198, 'potted plant': 170, 'bowl': 140, 'teddy bear': 111, 'kite': 110, 'book': 98, 'umbrella': 94, 'remote': 94, 'cake': 80, 'laptop': 78, 'bird': 77, 'motorcycle': 75, 'sandwich': 74, 'vase': 74, 'donut': 73, 'frisbee': 73, 'cat': 73, 'bicycle': 70, 'couch': 68, 'wine glass': 66, 'stop sign': 62, 'pizza': 61, 'traffic light': 58, 'handbag': 54, 'airplane': 54, 'tennis racket': 54, 'boat': 53, 'toothbrush': 52, 'bed': 50, 'baseball bat': 49, 'baseball glove': 42, 'backpack': 41, 'dining table': 40, 'scissors': 34, 'suitcase': 31, 'bench': 30, 'apple': 28, 'skateboard': 27, 'bus': 26, 'horse': 26, 'keyboard': 23, 'orange': 22, 'mouse': 22, 'refrigerator': 21, 'surfboard': 20, 'oven': 20, 'microwave': 19, 'banana': 18, 'train': 16, 'hot dog': 15, 'toilet': 14, 'broccoli': 14, 'bear': 12, 'fork': 11, 'sheep': 11, 'kn

Creating vector representation and saving it to data/ in CSV format

In [3]:
import pandas as pd

df = pd.DataFrame.from_dict(objects_detected).transpose()
df = df.fillna(0)
df = df.astype('int32')

df.to_csv("data/thumbnails_objects_vectors.csv")

Notatka:
Może wagi wilkością bboxów

Można więcej miar dać kolorów itd

może rozpoznawanie emocji na twarzach